In [1]:
badscript = '''contract get_fib_n(n:int){
    let last_fib_calculated : int = 0;

    entry get_fib(n: int){
        let result: int = fib(n);
        last_fib_calculated = result;
        let a : int = b;
    }

    func fib(n: int) : int{
        let x : int = last_fib_calculated;
        if (n <= 1) {
            return n;
        }
        else {
            
            let a: int = n - 1;
            let b: int = n - 2;
            return fib(a) + fib(b);
        }
    }

    func fib(n: int) : int{
        let x : int = last_fib_calculated;
    }
}'''

In [2]:
script = '''
contract get_fib_n(n:int){
    let last_fib_calculated: int = 0;

    if (n <= 1) {
            return n;
        }
    
    func fib(n: int) : int{
      
        else {
            let a: int = n - 1;
            let b: int = n - 2;
            return fib(a) + fib(b);
        }
    
    }
    while(n > 0){
        let a : int = 1;
    }
    entry get_fib_n(n: int){
        let result: int = fib(n);
        last_fib_calculated = result;
        return last_fib_calculated;
        
    }
    
}
'''

In [3]:
from lexer.sly_lexer import TzScriptLexer, process_lexer_tokens 
from lexer.lex_token import Token
from parser.tzscript_grammar import TZSCRIPT_GRAMMAR,idx, num, typex, contract, ifx, elsex,equal, plus, minus, star, div,semi, colon, comma, dot, opar, cpar, ocur, ccur,let, func,entry,lessthanequal,greaterthanequal, iniquelaty, lessthan,greaterthan,equalequal, returnx
from parser.slr_parser import SLR1Parser, build_slr_ast

lexer = TzScriptLexer()
lexer_tokens = list(lexer.tokenize(script))
tokens = process_lexer_tokens(lexer_tokens)

parser = SLR1Parser(TZSCRIPT_GRAMMAR, verbose=True)

terminals = [token.token_type for token in tokens]
derivation = parser(terminals, True)

0 	 <program> -> .contractid(<param-list>){<stat_list>}, 
	 S' -> .<program>,  

1 	 <program> -> contract.id(<param-list>){<stat_list>},  

2 	 <program> -> contractid.(<param-list>){<stat_list>},  

3 	 <param> -> .id:type, 
	 <program> -> contractid(.<param-list>){<stat_list>}, 
	 <param-list> -> .<param>, 
	 <param-list> -> .<param>,<param-list>,  

4 	 <program> -> contractid(<param-list>.){<stat_list>},  

5 	 <program> -> contractid(<param-list>).{<stat_list>},  

6 	 <return-stat> -> .return<expr>;, 
	 <program> -> contractid(<param-list>){.<stat_list>}, 
	 <stat> -> .<while-stat>, 
	 <let-var>> -> .letid:type=<expr>;, 
	 <stat_list> -> .<stat>, 
	 <stat> -> .<else-stat>, 
	 <stat> -> .<let-var>>, 
	 <stat> -> .<def-func>, 
	 <stat> -> .<return-stat>, 
	 <stat_list> -> .<stat_list><stat>, 
	 <if-stat> -> .if(<expr>){<stat_list>}, 
	 <stat> -> .<def-entry>, 
	 <stat> -> .<var-call>, 
	 <def-entry> -> .entryid(<param-list>){<stat_list>}, 
	 <def-func> -> .funcid(<param-list>):typ

In [4]:
productions, operations = derivation


from visitors.string_rep_visitor import FormatVisitor
ast = build_slr_ast(productions, operations, tokens)
format = FormatVisitor()
print(format.visit(ast))

\__ProgramNode: contract node.idx(n : int) [<stat>; ... <stat>;]
	\__AttrDeclarationNode: n : int
		\__VarDeclarationNode: let last_fib_calculated = <expr> : int
			\__ ConstantNumNode: 0
		\__IfNode: if <expr> then [<stat>; ... <stat>;]
			\__ LessThanEqualNode
				\__ VariableNode: n
				\__ ConstantNumNode: 1
			\__ReturnStatementNode: return <expr>
				\__ VariableNode: n
		\__FuncDeclarationNode: def fib(n : int) : int
				\__ElseNode: else [<stat>; ... <stat>;]
					\__VarDeclarationNode: let a = <expr> : int
						\__ MinusNode
							\__ VariableNode: n
							\__ ConstantNumNode: 1
					\__VarDeclarationNode: let b = <expr> : int
						\__ MinusNode
							\__ VariableNode: n
							\__ ConstantNumNode: 2
					\__ReturnStatementNode: return <expr>
						\__ PlusNode
							\__CallNode: fib(<expr>, ..., <expr>)
								\__ VariableNode: a
							\__CallNode: fib(<expr>, ..., <expr>)
								\__ VariableNode: b
		\__WhileNode: while <expr> then [<stat>; ... <stat>;]
			\__ Great

In [5]:
from visitors.semantic_check_visitor import SemanticCheckerVisitor
semantic_checker = SemanticCheckerVisitor()
semantic_checker.visit(ast)


[('If statement is not allowed in main level',
  <parser.tzscript_ast.IfNode at 0x1046be850>),
 ('Invalid variable n', <parser.tzscript_ast.VariableNode at 0x104567790>),
 'Return statement is not allowed in a funcion',
 ('Invalid variable n', <parser.tzscript_ast.VariableNode at 0x10432b810>),
 ('Else statement is not allowed without if statement',
  <parser.tzscript_ast.ElseNode at 0x10885f810>),
 'Return statement is not allowed in a funcion',
 'While statement is not allowed in main level',
 ('Invalid variable n', <parser.tzscript_ast.VariableNode at 0x1046b9950>),
 'Return statement is not allowed in a funcion',
 ('Invalid variable n', <parser.tzscript_ast.VariableNode at 0x104567790>),
 'Return statement is not allowed in a funcion',
 ('Invalid variable n', <parser.tzscript_ast.VariableNode at 0x10432b810>),
 'Function fib is defined after entry point',
 'Return statement is not allowed in a funcion',
 ('Invalid variable n', <parser.tzscript_ast.VariableNode at 0x1046b9950>),
 'R